In [4]:
class hi:
    def __init__(self, name):
        self.name = name
    def __str__(self) -> str:
        return self.name
    def __repr__(self) -> str:
        return self.name

a = hi("hello")
type(a)("oi")
a.__class__

__main__.hi

In [1]:
import pandas as pd
import numpy as np

In [7]:
instance_explanation_metrics = pd.DataFrame(columns=["method", "faithfullness_correlation", "sensitivity", "complexity"]).set_index("method")
instance_explanation_metrics.loc["shap"] = [0.9, 0.8, 0.7]
instance_explanation_metrics.loc["lime"] = [0.7, 0.6, 0.5]
instance_explanation_metrics.loc["anchor"] = [0.8, 0.7, 0.6]

display(instance_explanation_metrics)

,faithfullness_correlation,sensitivity,complexity
method,,,
shap,0.9,0.8,0.7
lime,0.7,0.6,0.5
anchor,0.8,0.7,0.6


In [21]:
def get_average_metric_rank(config_results: list[pd.DataFrame], is_more_better: list[bool]):
    ranks = []

    for instance_result in config_results:
        # instance_result is a dataframe with the following structure:
        # columns: each metric name
        # rows: each explainer name
        instance_rank = []
        for col_i, is_better in enumerate(is_more_better):
            if is_better:
                instance_rank.append(instance_result.iloc[:, col_i].rank(ascending=False))
            else:
                instance_rank.append(instance_result.iloc[:, col_i].rank(ascending=True))
                
        instance_rank = pd.concat(instance_rank, axis=1)
        ranks.append(instance_rank)
    
    avg_ranks = pd.concat(ranks).groupby(level=0).mean()
    
    return avg_ranks

In [25]:
get_average_metric_rank([instance_explanation_metrics], [True, True, False])

,faithfullness_correlation,sensitivity,complexity
method,,,
anchor,2.0,2.0,2.0
lime,3.0,3.0,1.0
shap,1.0,1.0,3.0


In [14]:
# Rank each column in descending order
ranked_metrics = instance_explanation_metrics["sensitivity"].rank(ascending=False).values
display(ranked_metrics)

array([1., 3., 2.])

In [15]:
instance_explanation_metrics.to_numpy()

array([[0.9, 0.8, 0.7],
       [0.7, 0.6, 0.5],
       [0.8, 0.7, 0.6]])

In [2]:
exp1 = pd.DataFrame(columns=["feature", "score"]).set_index("feature")
exp2 = pd.DataFrame(columns=["feature", "score"]).set_index("feature")

exp1.loc["feature1"] = [0.1]
exp1.loc["feature2"] = [0.2]
exp1.loc["feature3"] = [0.3]

exp2.loc["feature2"] = [0.2]
exp2.loc["feature3"] = [0.3]
exp2.loc["feature1"] = [0.4]

exp1.reset_index(inplace=True)
exp2.reset_index(inplace=True)

display(exp1)
display(exp2)

,feature,score
0,feature1,0.1
1,feature2,0.2
2,feature3,0.3


,feature,score
0,feature2,0.2
1,feature3,0.3
2,feature1,0.4


In [6]:
subset = ["feature1", "feature2"]
subset_g_x = exp1[exp1["feature"].isin(subset)]
display(subset_g_x["score"].values[0])

0.1

In [8]:
# align the two explanations
exp2 = exp2.set_index('feature').loc[exp1['feature']].reset_index()
display(exp1)
display(exp2)

,feature,score
0,feature1,0.1
1,feature2,0.2
2,feature3,0.3


,feature,score
0,feature1,0.4
1,feature2,0.2
2,feature3,0.3


In [2]:
a = {"a": 1, "b": 2}
b = {"d": 1, "e": 2, **a}
b

{'d': 1, 'e': 2, 'a': 1, 'b': 2}

In [15]:
from xai_agg.tools import *
ranking = get_ranking_df(exp1)
(1 / ranking["Rank"]).sum()

1.8333333333333333